In [1]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib, glob
import cartopy.crs as ccrs
from scipy.signal import savgol_filter
import string
from cartopy.util import add_cyclic_point
from matplotlib import colors

%run ./get_fixed_variables.ipynb # load plotting functions and fixed vars

In [2]:
## load surface temperature [K]
FSNS_8xco2_strato = np.load('/glade/campaign/univ/uhar0013/asalazar/saved_data/FSNS_8xco2_strato.npy')
FSNS_8xco2_nostrato = np.load('/glade/campaign/univ/uhar0013/asalazar/saved_data/FSNS_8xco2_nostrato.npy')
FSNS_1xco2_strato = np.load('/glade/campaign/univ/uhar0013/asalazar/saved_data/FSNS_PI_strato.npy')
FSNS_1xco2_nostrato = np.load('/glade/campaign/univ/uhar0013/asalazar/saved_data/FSNS_PI_nostrato.npy')


FLNS_8xco2_strato = np.load('/glade/campaign/univ/uhar0013/asalazar/saved_data/FLNS_8xco2_strato.npy')
FLNS_8xco2_nostrato = np.load('/glade/campaign/univ/uhar0013/asalazar/saved_data/FLNS_8xco2_nostrato.npy')
FLNS_1xco2_strato = np.load('/glade/campaign/univ/uhar0013/asalazar/saved_data/FLNS_PI_strato.npy')
FLNS_1xco2_nostrato = np.load('/glade/campaign/univ/uhar0013/asalazar/saved_data/FLNS_PI_nostrato.npy')


LHFLX_8xco2_strato = np.load('/glade/campaign/univ/uhar0013/asalazar/saved_data/LHFLX_8xco2_strato.npy')
LHFLX_8xco2_nostrato = np.load('/glade/campaign/univ/uhar0013/asalazar/saved_data/LHFLX_8xco2_nostrato.npy')
LHFLX_1xco2_strato = np.load('/glade/campaign/univ/uhar0013/asalazar/saved_data/LHFLX_PI_strato.npy')
LHFLX_1xco2_nostrato = np.load('/glade/campaign/univ/uhar0013/asalazar/saved_data/LHFLX_PI_nostrato.npy')



SHFLX_8xco2_strato = np.load('/glade/campaign/univ/uhar0013/asalazar/saved_data/SHFLX_8xco2_strato.npy')
SHFLX_8xco2_nostrato = np.load('/glade/campaign/univ/uhar0013/asalazar/saved_data/SHFLX_8xco2_nostrato.npy')
SHFLX_1xco2_strato = np.load('/glade/campaign/univ/uhar0013/asalazar/saved_data/SHFLX_PI_strato.npy')
SHFLX_1xco2_nostrato = np.load('/glade/campaign/univ/uhar0013/asalazar/saved_data/SHFLX_PI_nostrato.npy')


In [3]:
year_to_avg = 5
FLNS_diff_1xco2 = yearly_avg(FLNS_1xco2_nostrato,year_to_avg)-yearly_avg(FLNS_1xco2_strato,year_to_avg)
FSNS_diff_1xco2 = yearly_avg(FSNS_1xco2_nostrato,year_to_avg)-yearly_avg(FSNS_1xco2_strato,year_to_avg)
LHFLX_diff_1xco2 = yearly_avg(LHFLX_1xco2_nostrato,year_to_avg)-yearly_avg(LHFLX_1xco2_strato,year_to_avg)
SHFLX_diff_1xco2 = yearly_avg(SHFLX_1xco2_nostrato,year_to_avg)-yearly_avg(SHFLX_1xco2_strato,year_to_avg)

FLNS_diff_8xco2 = yearly_avg(FLNS_8xco2_nostrato,year_to_avg)-yearly_avg(FLNS_8xco2_strato,year_to_avg)
FSNS_diff_8xco2 = yearly_avg(FSNS_8xco2_nostrato,year_to_avg)-yearly_avg(FSNS_8xco2_strato,year_to_avg)
LHFLX_diff_8xco2 = yearly_avg(LHFLX_8xco2_nostrato,year_to_avg)-yearly_avg(LHFLX_8xco2_strato,year_to_avg)
SHFLX_diff_8xco2 = yearly_avg(SHFLX_8xco2_nostrato,year_to_avg)-yearly_avg(SHFLX_8xco2_strato,year_to_avg)


In [4]:
conv = 1000*60*60*24
LANDMASK_nan = LANDMASK*np.nan
#LANDMASK_nan[yearly_avg(SOILLIQ_8xco2[:,0,:,:], 30) < 1e10] = 1
S2_vars = np.array([[FSNS_diff_8xco2, FSNS_diff_1xco2],
                         [FLNS_diff_8xco2, FLNS_diff_1xco2],
                    [LHFLX_diff_8xco2, LHFLX_diff_1xco2],
                   [SHFLX_diff_8xco2, SHFLX_diff_1xco2],
                   [FSNS_diff_8xco2-FLNS_diff_8xco2-LHFLX_diff_8xco2-SHFLX_diff_8xco2, FSNS_diff_1xco2-FLNS_diff_1xco2-LHFLX_diff_1xco2-SHFLX_diff_1xco2]])
levels_list = [[np.linspace(-30,30,33),np.linspace(-30,30,33)],
               [np.linspace(-30,30,33),np.linspace(-30,30,33)],
               [np.linspace(-30,30,33),np.linspace(-30,30,33)],
                        [np.linspace(-30,30,33),np.linspace(-30,30,33)],
               [np.linspace(-30,30,33),np.linspace(-30,30,33)]]
cmap_list = np.array([['RdBu_r','RdBu_r'],
                      ['RdBu_r','RdBu_r'],
                      ['RdBu_r','RdBu_r'],
                      ['RdBu_r','RdBu_r'],
                     ['RdBu_r','RdBu_r']])
title_list = np.array([['$\\Delta$ Net Surface Shortwave (Wm$^{-2}$) \n in 8xCO$_2$-locked $\\it{no}$-$\\it{strato} - \\it{strato}$', '$\\Delta$ Net Surface Shortwave (Wm$^{-2}$) \n in 1xCO$_2$-locked $\\it{no}$-$\\it{strato} - \\it{strato}$'],
                       ['$\\Delta$ Net Surface Longwave (Wm$^{-2}$) \n in 8xCO$_2$, $\\it{no}$-$\\it{strato} - \\it{strato}$', '$\\Delta$ Net Surface Longwave (Wm$^{-2}$) \n in 1xCO$_2$, $\\it{no}$-$\\it{strato} - \\it{strato}$'],
                       ['$\\Delta$ LH Flux (Wm$^{-2}$) \n in 8xCO$_2$, $\\it{no}$-$\\it{strato} - \\it{strato}$','$\\Delta$ LH Flux (Wm$^{-2}$) \n in 1xCO$_2$, $\\it{no}$-$\\it{strato} - \\it{strato}$'],
                      ['$\\Delta$ SH Flux (Wm$^{-2}$) \n in 8xCO$_2$, $\\it{no}$-$\\it{strato} - \\it{strato}$','$\\Delta$ SH Flux (Wm$^{-2}$) \n in 1xCO$_2$, $\\it{no}$-$\\it{strato} - \\it{strato}$'],
                      ['$\\Delta$ SEB (Wm$^{-2}$) \n in 8xCO$_2$, $\\it{no}$-$\\it{strato} - \\it{strato}$','$\\Delta$ SEB (Wm$^{-2}$) \n in 1xCO$_2$, $\\it{no}$-$\\it{strato} - \\it{strato}$']])


text_list = np.array([['A','B'],['C','D'],['E','F'],['G','H'],['I','J']])


In [ ]:
extend = 'both'
shrink = 0.7
make_subplots(S2_vars, levels_list,cmap_list, title_list, 'Figure_S5', text_list, (12,15))